In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 20694151
paper_name = 'saleem_aitchison_2010' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [21]:
original_data = pd.read_excel('raw_data/journal.pone.0011953.s004.xlsx', sheet_name='1', skiprows=4)

In [22]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4049 x 9


In [23]:
original_data.head()

,Position,Accession,Gene,I.,II.,Unnamed: 5,I..1,Flow Cytometry Analysis,Unnamed: 8
0,101F10,YLL002W,REM50,-1.925611,H,NaN,NaN,Standard scores of the regression modeled valu...,NaN
1,101F11,YLL005C,SPO75,0.263767,G,NaN,NaN,NaN,NaN
2,101F12,YLL010C,PSR1,0.807619,A,NaN,II.,Confocal Microscopy Analysis,NaN
3,101F9,YLL001W,DNM1,1.061771,H,NaN,NaN,Phenotype,Key
4,101G1,YLL012W,YLL012W,1.524271,H,NaN,NaN,Normal,A


In [24]:
original_data['orf'] = original_data['Accession'].astype(str)

In [25]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [26]:
original_data.loc[original_data['orf']=='YYKL138C','orf'] = 'YKL138C'

In [27]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [28]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Position, Accession, Gene, I., II., Unnamed: 5, I..1, Flow Cytometry Analysis, Unnamed: 8, orf]
Index: []


In [29]:
original_data.set_index('orf', inplace=True)

In [30]:
original_data = original_data[['I.','II.']].copy()

In [31]:
t = pd.pivot_table(original_data, index='orf', columns='II.', values='I.')

In [33]:
t[t.notnull()] = 1
t[t.isnull()] = 0

In [35]:
t['BC'] = t['C'] - t['B']
t['DE'] = t['E'] - t['D']
t['J'] = -t['J']
t['KL'] = -t['K'] - 0.5 * t['L']

In [37]:
original_data2 = t[['BC','DE','J','KL']].copy()

In [38]:
original_data = original_data.join(original_data2, how='outer')

In [40]:
original_data.drop(columns='II.', inplace=True)

In [41]:
original_data = original_data.groupby(original_data.index).mean()

In [42]:
original_data.shape

(4028, 5)

# Prepare the final dataset

In [43]:
data = original_data.copy()

In [44]:
dataset_ids = [11830, 11831, 11832, 11834, 11835]
datasets = datasets.reindex(index=dataset_ids)

In [45]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [46]:
data.head()

dataset_id,11830,11831,11832,11834,11835
data_type,value,value,value,value,value
orf,,,,,
YAL012W,1.082880,0.0,0.0,0.0,0.0
YAL024C,0.119974,0.0,0.0,0.0,0.0
YAL027W,0.080831,-1.0,0.0,0.0,0.0
YAL039C,0.228864,0.0,0.0,0.0,0.0
YAL064C-A,0.488131,0.0,0.0,0.0,0.0


## Subset to the genes currently in SGD

In [47]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 23


In [48]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,11830,11831,11832,11834,11835
,data_type,value,value,value,value,value
gene_id,orf,,,,,
10,YAL012W,1.082880,0.0,0.0,0.0,0.0
22,YAL024C,0.119974,0.0,0.0,0.0,0.0
25,YAL027W,0.080831,-1.0,0.0,0.0,0.0
37,YAL039C,0.228864,0.0,0.0,0.0,0.0
1867,YAL064C-A,0.488131,0.0,0.0,0.0,0.0


# Normalize

In [49]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [50]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [51]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,11830,11831,11832,11834,11835,11830,11831,11832,11834,11835
,data_type,value,value,value,value,value,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,
10,YAL012W,1.082880,0.0,0.0,0.0,0.0,0.640505,0.087235,0.113045,0.0,0.0
22,YAL024C,0.119974,0.0,0.0,0.0,0.0,-0.264711,0.087235,0.113045,0.0,0.0
25,YAL027W,0.080831,-1.0,0.0,0.0,0.0,-0.301509,-8.549033,0.113045,0.0,0.0
37,YAL039C,0.228864,0.0,0.0,0.0,0.0,-0.162345,0.087235,0.113045,0.0,0.0
1867,YAL064C-A,0.488131,0.0,0.0,0.0,0.0,0.081389,0.087235,0.113045,0.0,0.0


# Print out

In [52]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [53]:
from IO.save_data_to_db3 import *

In [54]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/5 [00:00<?, ?it/s]

Deleting all datasets for PMID 20694151...
Inserting the new data...


100%|██████████| 5/5 [00:32<00:00,  6.51s/it]

Updating the data_modified_on field...
